Memory on LLM Chain

LLM Chain is off-the-shelf chain, which is for general purpose.
Customed Chain is more preferred 

In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain

from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
        temperature=0.9,
        callbacks=[StreamingStdOutCallbackHandler()]
    )
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=50,
    return_messages=True, # False: Just String. True: return as Message Class
    memory_key="chat_history" # To connect memory with Template->Prompt->Chain.
)

# template = """
#     You are a kind AI talking to a human

#     {chat_history}
#     Human: {question}
#     You: 
# """
# prompt = PromptTemplate.from_template(template)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a kind AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), # unlimited amount of messages can be stored
    ("human", "{question}")
])

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True # For debug - Prompt log
)

chain.predict(question="My name is Minji")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a kind AI talking to a human
Human: My name is Minji

> Finished chain.


"It's nice to meet you, Minji! How can I assist you today?"

In [6]:
chain.predict(question="I live in a Halloween Town")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a kind AI talking to a human
Human: My name is Minji
AI: It's nice to meet you, Minji! How can I assist you today?
Human: I live in a Halloween Town

> Finished chain.


'That sounds like a fun and festive place to live! Do you have any exciting plans for Halloween this year in your Halloween Town?'

In [7]:
chain.predict(question="It is very cozy like sleeping in a coffin.. do you remember where am I living?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a kind AI talking to a human
System: The human introduces themselves as Minji. The AI responds warmly and asks how it can assist Minji today.
Human: I live in a Halloween Town
AI: That sounds like a fun and festive place to live! Do you have any exciting plans for Halloween this year in your Halloween Town?
Human: It is very cozy like sleeping in a coffin.. do you remember where am I living?

> Finished chain.


'Yes, you mentioned earlier that you live in a Halloween Town. It sounds like a unique and cozy place, similar to the feeling of sleeping in a coffin! Is there anything specific you enjoy about living in this Halloween-themed environment?'

Memory on Customed Chain
Prompt and memories are same.

RunnablePassthrough - the class allows you to run the function before the prompt is formatted.



In [13]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
        temperature=0.9
    )
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=50,
    return_messages=True, # False: Just String. True: return as Message Class
    memory_key="chat_history" # To connect memory with Template->Prompt->Chain.
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a kind AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), # unlimited amount of messages can be stored
    ("human", "{question}")
])

# chain = LLMChain(
#     llm = llm,
#     memory = memory,
#     prompt = prompt,
#     verbose = True #  Prompt log
# )

## Without runnable pass through
# chain = prompt | llm
# chain.invoke({ # chain호출
#     "chat_history" : memory.load_memory_variables({})["chat_history"], // This can be conplicated
#     "question": "My name is Minji"
# })


def load_memory(input):
    print(input) # question
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
chain.invoke({ # chain호출
    "question": "My name is Minji"
})# first input of the chain: dictionary

{'question': 'My name is Minji'}


AIMessage(content="Hello Minji, it's nice to meet you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 25, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d96a3773-7573-480e-9175-2def8f3a1933-0')